In [17]:
# import numpy as np
#
# # z1, y1, x1, z2, y2, x2
# a = np.array([0, 0, 1, 6, 7, 9])
# b = np.array([2, 5, 3, 10, 10, 10])
#
# zA = max(a[0], b[0])
# yA = max(a[1], b[1])
# xA = max(a[2], b[2])
# zB = max(a[3], b[3])
# yB = max(a[4], b[4])
# xB = max(a[5], b[5])
#
# interArea = max(0, xB - xA) * max(0, yB - yA) * max(0, zB - zA)
#
# boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
# boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])
#
# iou = interArea / float(boxAArea + boxBArea - interArea)

In [18]:
def iou(a, b):
    zA = tf.maximum(a[0], b[0])
    yA = tf.maximum(a[1], b[1])
    xA = tf.maximum(a[2], b[2])
    zB = tf.maximum(a[3], b[3])
    yB = tf.maximum(a[4], b[4])
    xB = tf.maximum(a[5], b[5])

    interArea = tf.maximum(0, xB - xA) * tf.maximum(0, yB - yA) * tf.maximum(0, zB - zA)

    boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
    boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])

    return interArea / (boxAArea + boxBArea - interArea)


In [19]:
import tensorflow as tf

a = tf.constant([0, 0, 1, 6, 7, 9])
b = tf.constant([2, 5, 3, 10, 10, 10])

iou = iou(a, b)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  iou_value = iou.eval()

In [30]:
tf.concat([a[:2], a[2:]], 0)

<tf.Tensor 'concat_5:0' shape=(6,) dtype=int32>

In [ ]:
boxes = tf.constant([
    [0, 0, 1, 6, 7, 9],
    [2, 5, 3, 10, 10, 10],
    [20, 50, 30, 100, 100, 100],
    [21, 40, 10, 330, 350, 360]
    ]
)

scores = tf.constant([
    0.2,
    0.4,
    0.1,
    0.6
])

boxes_prym = boxes[1:, :]

In [64]:
def iou(a, b):
    zA = tf.maximum(a[0], b[0])
    yA = tf.maximum(a[1], b[1])
    xA = tf.maximum(a[2], b[2])
    zB = tf.maximum(a[3], b[3])
    yB = tf.maximum(a[4], b[4])
    xB = tf.maximum(a[5], b[5])

    interArea = tf.maximum(0, xB - xA) * tf.maximum(0, yB - yA) * tf.maximum(0, zB - zA)

    boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
    boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])

    return interArea / (boxAArea + boxBArea - interArea)


def iou_evaluation(n, boxes, scores, highest_score_box, nms_threshold):
    iou_value = iou(highest_score_box, boxes[n])
    (boxes, scores) = tf.cond(
        tf.greater(iou_value, nms_threshold),
        lambda: (
            tf.cast(tf.concat([boxes[:n], boxes[n:]], 0), tf.float32),
            tf.cast(tf.concat([scores[:n], scores[n:]], 0), tf.float32)
        ),
        lambda: (tf.cast(boxes, tf.float32), tf.cast(scores, tf.float32))
    )
    return [tf.add(n, 1), boxes, scores, highest_score_box, nms_threshold]


def nms_step(n, boxes, scores, sorted_args, nms_threshold):
    highest_score_arg = sorted_args[0]
    highest_score_box = boxes[highest_score_arg]
    print(highest_score_box)
    number_of_processed = tf.constant(0)
    boxes_size = tf.shape(boxes)[0]
    tf.while_loop(
        lambda n, b, s, h, t: n < boxes_size,
        lambda n, b, s, h, t: iou_evaluation(n, b, s, h, t),
        [
            number_of_processed,
            tf.concat([boxes[:highest_score_arg], boxes[highest_score_arg:]], 0),
            tf.concat([scores[:highest_score_arg], scores[highest_score_arg:]], 0),
            highest_score_box,
            nms_threshold
        ],
        shape_invariants=[
            number_of_processed.get_shape(),
            tf.TensorShape([None, 6]),
            tf.TensorShape([None]),
            tf.TensorShape([None]),
            nms_threshold.get_shape()
        ]
    )
    return [tf.add(n, 1), tf.constant([[2, 0, 2, 3, 4, 1]]), scores, sorted_args, nms_threshold]


def nms(boxes, scores, proposal_count=4, nms_threshold=0.5):
    print(boxes)
    print(scores)
    print(proposal_count)
    print(nms_threshold)

    number_of_processed = tf.constant(0)
    boxes_size = tf.shape(boxes)[0]
    sorted_args = tf.argsort(scores, direction='DESCENDING')
    nms_threshold = tf.constant(nms_threshold, dtype=tf.float64)

    result = tf.while_loop(
        lambda n, b, s, sa, t: n < boxes_size,
        lambda n, b, s, sa, t: nms_step(n, b, s, sa, t),
        [number_of_processed, boxes, scores, sorted_args, nms_threshold],
        shape_invariants=[
            number_of_processed.get_shape(),
            tf.TensorShape([None, 6]),
            tf.TensorShape([None]),
            tf.TensorShape([None]),
            nms_threshold.get_shape()
        ]
    )

nms(boxes, scores)


Tensor("Const_2:0", shape=(4, 6), dtype=int32)
Tensor("Const_3:0", shape=(4,), dtype=float32)
4
0.5
Tensor("while_47/strided_slice_1:0", shape=(6,), dtype=int32)
